In [ ]:
import ta
import numpy as np
import pandas as pd
import random

import warnings
# Turn off all warnings
warnings.filterwarnings("ignore")

In [ ]:
import ccxt
import pandas as pd

# Fetch OHLCV data from Kraken
def fetch_ohlcv_data(start_date):
    exchange = ccxt.kraken()
    symbol = 'BTC/AUD'
    timeframe = '1d'
    since = exchange.parse8601(start_date)
    ohlcv_data = exchange.fetch_ohlcv(symbol, timeframe, since, limit=None)
    
    # Convert the data to a Pandas DataFrame and set column names
    ohlcv_df = pd.DataFrame(ohlcv_data, columns=['timestamp', 'o', 'h', 'l', 'c', 'v'])
    # Convert the timestamp to a human-readable datetime format
    ohlcv_df['timestamp'] = pd.to_datetime(ohlcv_df['timestamp'], unit='ms')

    return ohlcv_df

ohlcv_data = fetch_ohlcv_data("2021-01-01T00:00:00Z")

print(ohlcv_data[:3])

In [ ]:
from ta.momentum import RSIIndicator
from ta.trend import SMAIndicator, MACD
# Define the bot's functions
def buy_trigger(t, P, indicators):
    rsi_value = indicators.loc[t, 'rsi']
    rsi_threshold = P[0]
    macd_value = indicators.loc[t, 'macd']
    signal_value = indicators.loc[t, 'signal']

    return (rsi_value < rsi_threshold) or (macd_value > signal_value)

def sell_trigger(t, P, indicators):
    close = ohlcv_data.loc[t, 'c']
    sma_value = indicators.loc[t, 'sma']


    return (close > sma_value)

def trading_bot(P, ohlcv_data):
    # Create an RSI indicator with a period of 14
    rsi_period = 20
    rsi_indicator = RSIIndicator(ohlcv_data['c'], window=rsi_period)
    rsi = rsi_indicator.rsi()
    # Add the RSI values to the ohlcv_data DataFrame
    
    # Create an SMA indicator with a period of 20
    sma_period = 20
    sma_indicator = SMAIndicator(ohlcv_data['c'], window=sma_period)
    sma = sma_indicator.sma_indicator()

    # Add the SMA values to the ohlcv_data DataFrame
    ohlcv_data['sma'] = sma

    # Create a MACD indicator with default parameters (12, 26, 9)
    macd_indicator = MACD(ohlcv_data['c'])
    macd = macd_indicator.macd()  # MACD line
    signal = macd_indicator.macd_signal()  # Signal line
    histogram = macd_indicator.macd_diff()  # Histogram (MACD - Signal)

    # Add the MACD values to the indicators DataFrame
    indicators = pd.DataFrame(index=ohlcv_data.index)
    indicators['rsi'] = rsi
    indicators['sma'] = sma
    indicators['macd'] = macd
    indicators['signal'] = signal

    n = len(ohlcv_data)
    positions = []
    bought = False

    fiat_money = 100
    btc_money = 0
    fee = 0.02 # Each buy or sell event costs 2% of current holdings.

    for t in range(1, n):
        close_price = ohlcv_data.loc[t, 'c']        

        if buy_trigger(t, P, indicators) and not bought and not sell_trigger(t, P, indicators):
            positions.append(('buy', t))
            btc_money = (fiat_money / close_price) * (1 - fee)
            fiat_money = 0
            bought = True
        elif sell_trigger(t, P, indicators) and bought and not buy_trigger(t, P, indicators):
            positions.append(('sell', t))
            fiat_money = (btc_money * close_price) * (1 - fee)
            btc_money = 0
            bought = False

    if bought:
        positions.append(('sell', n - 1))

    return positions, fiat_money, btc_money

P = [30, 70]  # Define the set of parameters for the bot (buy RSI threshold, sell RSI threshold)
positions, fiat_money, btc_money = trading_bot(P, ohlcv_data)
print(positions)
print(fiat_money, btc_money)

In [ ]:
import matplotlib.pyplot as plt

# Extract the timestamps, close prices, and buy/sell signals
timestamps = ohlcv_data.index
close_prices = ohlcv_data['c']
buy_signals = [t for action, t in positions if action == 'buy']
sell_signals = [t for action, t in positions if action == 'sell']

# Plot the price data
plt.plot(timestamps, close_prices, label='Close Price', color='blue', alpha=0.7)

# Plot the buy signals
plt.scatter([timestamps[t] for t in buy_signals],
            [close_prices[t] for t in buy_signals],
            color='g', s=25, marker="^", label='Buy Signal')

# Plot the sell signals
plt.scatter([timestamps[t] for t in sell_signals],
            [close_prices[t] for t in sell_signals],
            color='r', s=25, marker="v", label='Sell Signal')

# Customize the plot appearance

plt.xlabel('Time')
plt.ylabel('Close Price')
plt.title('BTC/AUD Trading Bot Buy/Sell Signals')
plt.legend()
plt.grid()

# Show the plot
plt.show()